In [ ]:
import json

In [ ]:
# this code block allows you to inspect 2 tweets and 2 retweets in the dataset
# it also saves all the unique tweets/retweets in the dictionary tweets
# this code will help you to understand the data
# feel free to print out more fields

lines=0
num_tweets=0
num_retweets=0
print_tweet = 0
print_retweet = 0
tweets = {}

#insert path and replace name of the file below as needed
with open("corona-out-3", "r") as f1:
    for line in f1:
        try:
            data = json.loads(line)
            lines = lines + 1
            if ( data['text'].startswith('RT') ):
                num_retweets += 1
                #print out some fields of one retweet 
                #note that you should look at other fields too
                if (print_retweet < 1):
                    print_retweet += 1
                    print('RETWEET\n','id--',data['id'],'text--',data['text'],'\n')
                    
                    print('User-- ',data['user'],'\n')
                  
                    print('retweetstatus--',data['retweeted_status'],'\n')
                  

            else:
                num_tweets += 1
                #print out some fields of one tweet 
                #note that you should look at other fields too
                if (print_tweet < 1):
                    print_tweet += 1
                    print('TWEET\n','id--',data['id'],'text--',data['text'],'\n')
                    print('User-- ',data['user'],'\n')
                    


            if (data['id_str'] not in tweets):
                tweets[data['id_str']] = data
            
        except:
            # if there is an error loading the json of the tweet, skip
            continue
            
print('num of lines=',lines, 'num of tweets=',num_tweets, 'num of retweets=',num_retweets)
print('num of unique tweets/retweets=',len(tweets.keys()))

In [ ]:
# this is a partial template for processing the dataset to store into the datastores.
# you will need to write functions for the psuedocode sections and invoke them here

#insert path and replace name of the file below as needed

datastores = {'tweets': {}, 'users': {}, 'metrics': {}, 'retweets': {}}

def tweet_already_seen(tweet_id, datastore):
    """
    Check if the tweet has been seen before.
    """
    return tweet_id in datastore['tweets']

def user_not_seen_before(user_id, datastore):
    """
    Check if the user has not been seen before.
    """
    return user_id not in datastore['users']

def add_user_to_table(user, datastore):
    """
    Add user to the user table.
    """
    datastore['users'][user['id']] = user

def update_datastores(data, datastore):
    """
    Update datastores with any metrics that you are tracking.
    """
    # Update tweet count
    datastore['metrics']['tweet_count'] = datastore['metrics'].get('tweet_count', 0) + 1

def update_retweet_info(data, datastore):
    """
    Update retweet information.
    """
    # Update retweet count
    datastore['metrics']['retweet_count'] = datastore['metrics'].get('retweet_count', 0) + 1
    

def add_tweet_to_datastore(tweet, datastore):
    """
    Add the new tweet to the datastore.
    """
    datastore['tweets'][tweet['id']] = tweet

def add_retweet_to_datastore(retweet, datastore):
    """
    Add the retweet to the datastore
    """
    datastore['retweets'][retweet['id']] = retweet
    
with open("corona-out-3", "r") as f1:
    for line in f1:
        try:
            data = json.loads(line)
            
            # Check if tweet has been seen before
            if tweet_already_seen(data['id'], datastores):
                continue
            
            user = data['user']
            
            # Check if user has not been seen before
            if user_not_seen_before(user['id'], datastores):
                add_user_to_table(user, datastores)
            
            # Update datastores with any metrics that you are tracking
            update_datastores(data, datastores)
            if data['text'].startswith('RT'):
                # Update retweet information
                add_retweet_to_datastore(data, datastores)
                # Note: You may not have an entry for the original tweet
                # if that is not in the dataset
            else:
                # Add the new tweet to datastore
                add_tweet_to_datastore(data, datastores)
                
        except:
            continue

HASHTAG : NOSQL: PRIMARY KEY - TAG

#tag3 - ID1

#tag3 - ID2

#tag1 - ID2

#tag1 - ID1

#tag2 - ID1


USER : NOSQL : PRIMARY : user_id

Search thru screen_name, name


In [ ]:
import json
import pymongo
from pymongo import MongoClient

# Connect to MongoDB
client = pymongo.MongoClient()  
db = client.db 
users_collection = db.user  
hashtags_collection = db.hashtags

try:
    users_collection.create_index([('id', 1)], unique=True)
except errors.OperationFailure as e:
    print(f"Error creating index: {e}")

def add_user_to_collection(user):
    """
    Add user to the users collection in MongoDB.
    """
    try:
        users_collection.insert_one(user)
    except errors.DuplicateKeyError:
        print("User already exists:", user['id'])
    except Exception as e:
        print("Error inserting user:", e)
        
def insert_hashtags(tweet_id, hashtags, type_of_tweet):
    """
    Insert hashtags along with tweet ID into the hashtags collection in MongoDB.
    """
    try:
        for hashtag in hashtags:
            hashtags_collection.insert_one({
                "hashtag": hashtag,
                "tweet_id": tweet_id,
                'type': type_of_tweet
            })
    except Exception as e:
        print("Error inserting hashtag:", e)
        
with open("corona-out-3", "r") as f1:
    for line in f1:
        try:
            # Extract tweet ID
            data = json.loads(line) 
            user = data['user'] 
            add_user_to_collection(user) # Capture user details, if not already present in User table
            tweet_id = data["id"]
            if data['text'].startswith('RT'):
                # Retweets
                # Capture retweet-hashtag details
                # Capture original tweet-hashtag details, if not already present in Hashtag table
                if data['retweeted_status']:
                    hashtags = [hashtag["text"] for hashtag in data['retweeted_status']['extended_tweet']['entities']['hashtags']]
                    insert_hashtags(data['retweeted_status']['id'], hashtags, 'RT')
                    add_user_to_collection(data['retweeted_status']['user']) # add in the user if it doesn't exist already
            else:
                # Tweets
                # Extract hashtags 
                # Capture hashtag details 
                hashtags = [hashtag["text"] for hashtag in data["entities"]["hashtags"]]
                insert_hashtags(tweet_id, hashtags, 'OG')
        except Exception as e:
            continue

In [ ]:
cursor = users_collection.find()
for document in cursor:
    print(document)

In [ ]:
cursor = hashtags_collection.find()
for document in cursor:
    print(document)

In [ ]:
db.hashtags.drop()

In [ ]:
db.users.drop()